<a href="https://colab.research.google.com/github/otanet/RecSys/blob/master/6_Hybrid_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Recommenders: Content-based model + collaboratibe filtering

#### 分析処理のワークフローは以下のとおり
#### 1. タイトル(title)とユーザー(user)を取得
#### 2. content-based modelで25個の最も類似度が高い映画を算出
#### 3. 2.の25個の映画をcollavorative filterにかけて、レイティングを予測
#### 4. 最も高いレイティングの上位10個を戻り値で返す


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My\ Drive/data

[Errno 2] No such file or directory: '/content/drive/My Drive/data'
/content


In [0]:
ls

cosine_sim.csv      credits.csv   metadata_clean.csv  movies_metadata.csv
cosine_sim_map.csv  keywords.csv  movie_ids.csv       ratings_small.csv


In [0]:
!pip install scikit-surprise

In [0]:
# ライブラリーのインポート
import numpy as np
import pandas as pd

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import PredefinedKFold

from surprise.model_selection import cross_validate

from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

In [0]:
#ファイル(the cosine_sim matrix)をインポート
cosine_sim = pd.read_csv('cosine_sim.csv')

In [0]:
#コサイン類似度の行列を読みこむ
cosine_sim_map = pd.read_csv('cosine_sim_map.csv', header=None)

#PandasのSeries型に変換
cosine_sim_map = cosine_sim_map.set_index(0)
cosine_sim_map = cosine_sim_map[1]

In [0]:
#協調フィルタリングを基にした特異値分解の適用
from surprise import SVD, Reader, Dataset

reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#data.split(n_folds=5)
trainset = data.build_full_trainset()
#algo.fit(trainset)
svd = SVD()
#trainset = data.build_full_trainset()
svd.fit(trainset)
#svd.train(trainset)


#特異値分解(SVD)の処理
#algo = SVD()

# N=5のクロスバリデェーションを行う
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [0]:
#さらに、ファイルを読み込み、タイトルにIDを紐付ける
id_map = pd.read_csv('movie_ids.csv')
id_to_title = id_map.set_index('id')
title_to_id = id_map.set_index('title')

In [0]:
#メタデータとして、ファイルを読みこむ
smd = pd.read_csv('metadata_small.csv')

In [0]:
def hybrid(userId, title):
    #インデックスとコサイン類似度を抽出する
    idx = cosine_sim_map[title]
    
    #タイトルとIDを抽出し、TMDBIDとして保存
    tmdbId = title_to_id.loc[title]['id']
    
    #タイトルとmovieIDを抽出し保存
    movie_id = title_to_id.loc[title]['movieId']
    
    #コサイン類似度の行列からその類似度と映画ごとに対応するスコアを抽出する
    sim_scores = list(enumerate(cosine_sim[str(int(idx))]))
    
    #タプルのインデックスとスコアを降順にソートする
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #上位25個を取り出す 
    sim_scores = sim_scores[1:26]
    
    #リストとして保存する
    movie_indices = [i[0] for i in sim_scores]

    #5つのインデックスを変数movieに格納する
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    
    #特異値分解(SVD)を用いて予測する
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, id_to_title.loc[x]['movieId']).est)
    
    #予測値を降順にソートする
    movies = movies.sort_values('est', ascending=False)
    
    #映画10個を戻り値として返す
    return movies.head(10)

In [0]:
#アバターの場合のレコメンド映画上位10個
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
1011,The Terminator,4208.0,7.4,1984,218,3.207429
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.160871
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.142554
974,Aliens,3282.0,7.7,1986,679,3.094333
7705,Alice in Wonderland,8.0,5.4,1933,25694,2.938126
2834,Predator,2129.0,7.3,1987,106,2.904674
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.881617
922,The Abyss,822.0,7.1,1989,2756,2.877822
2014,Fantastic Planet,140.0,7.6,1973,16306,2.866388
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,2.782715


In [0]:
#アバターの場合のレコメンド映画上位10個
hybrid(2, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.983453
1011,The Terminator,4208.0,7.4,1984,218,3.846616
2014,Fantastic Planet,140.0,7.6,1973,16306,3.799311
974,Aliens,3282.0,7.7,1986,679,3.711100
2834,Predator,2129.0,7.3,1987,106,3.646443
7705,Alice in Wonderland,8.0,5.4,1933,25694,3.586773
4966,Hercules in New York,63.0,3.7,1969,5227,3.578447
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,3.551712
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.536026
7088,Star Wars: The Clone Wars,434.0,5.8,2008,12180,3.490607
